In [1]:
# virtual environment check. must be venv_Quant
import sys, os
print("Interpreter:", sys.executable)
print("In venv:", sys.prefix != getattr(sys, "base_prefix", sys.prefix))
print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
print("CONDA_DEFAULT_ENV:", os.environ.get("CONDA_DEFAULT_ENV"))

Interpreter: c:\Enzo_Files\25Wk37 Quant Trading\venv_Quant\Scripts\python.exe
In venv: True
VIRTUAL_ENV: C:\Enzo_Files\25Wk37 Quant Trading\venv_Quant
CONDA_DEFAULT_ENV: None


In [2]:
# !pip install plotly pandas numpy scikit-learn 
# !pip install requests
# !pip install nbformat

### Trading Strategy Simulation with Python (MetaTrader5-Style UI)

- Historical price chart (candlestick chart for Bitcoin price, using 10+ years of data if available).
- Real-time data integration (fetch recent BTC data from Binance API).
- Technical indicators (e.g., a moving average on the price chart).
- Trading strategy algorithms – we'll demonstrate Martingale and "Fish Tail" (anti-martingale) strategies.
- Trade position markers on the chart (buy/sell points).
- Performance tracking (profit/loss calculation for each strategy).


In [3]:
import requests
import pandas as pd
from datetime import datetime

# Binance API endpoint for historical klines (candles)
API_URL = "https://api.binance.com/api/v3/klines"
symbol = "BTCUSDT"
# symbol = "ETHUSDT"
# symbol = "SOLUSDT"
# symbol = "BNBUSDT"
# symbol = "DOGEUSDT"
# symbol = "ADAUSDT"
interval = "1d"  # 1-day interval for ~10 years of daily data

# Initialize parameters for data fetching loop
start_time = int(datetime(2015, 1, 1).timestamp() * 1000)  # start from 2015 (earlier than Binance launch)
all_data = []  # to collect raw kline data

# Fetch data in loops of 1000 candles until no more data
while True:
    params = {"symbol": symbol, "interval": interval, "startTime": start_time, "limit": 1000}
    response = requests.get(API_URL, params=params)
    data = response.json()
    if not data:
        break  # no more data returned
    all_data.extend(data)
    if len(data) < 1000:
        break  # last batch fetched
    # Set start_time to last candle's close time + 1 ms to get the next batch
    last_close = int(data[-1][6])
    start_time = last_close + 1

# Load data into a pandas DataFrame with proper columns
cols = ["open_time", "open", "high", "low", "close", "volume", 
        "close_time", "quote_asset_volume", "num_trades", 
        "taker_buy_base", "taker_buy_quote", "ignore"]
df = pd.DataFrame(all_data, columns=cols)

# Convert time columns to datetime and numeric types for prices/volumes
df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
numeric_cols = ["open", "high", "low", "close", "volume"]
df[numeric_cols] = df[numeric_cols].astype(float)

# Set the index to open_time (or close_time). We'll use open_time for chart x-axis.
df.set_index("open_time", inplace=True)
# Keep only relevant columns
df = df[["open", "high", "low", "close", "volume"]]

# Print data range and sample
print(f"Fetched BTC/USDT data from {df.index[0].date()} to {df.index[-1].date()} — {len(df)} days")
print(df.head(10))  # print first 3 rows as sample
print("...\n", df.tail(10))  # print last 3 rows as sample


Fetched BTC/USDT data from 2017-08-17 to 2025-09-09 — 2946 days
               open     high      low    close       volume
open_time                                                  
2017-08-17  4261.48  4485.39  4200.74  4285.08   795.150377
2017-08-18  4285.08  4371.52  3938.77  4108.37  1199.888264
2017-08-19  4108.37  4184.69  3850.00  4139.98   381.309763
2017-08-20  4120.98  4211.08  4032.62  4086.29   467.083022
2017-08-21  4069.13  4119.62  3911.79  4016.00   691.743060
2017-08-22  4016.00  4104.82  3400.00  4040.00   966.684858
2017-08-23  4040.00  4265.80  4013.89  4114.01  1001.136565
2017-08-24  4147.00  4371.68  4085.01  4316.01   787.418753
2017-08-25  4316.01  4453.91  4247.48  4280.68   573.612740
2017-08-26  4280.71  4367.00  4212.41  4337.44   228.108068
...
                  open       high        low      close       volume
open_time                                                          
2025-08-31  108816.33  109480.02  108076.93  108246.35   9489.51596
2025-09

### 2. Visualize Price History with Technical Indicator
- Initial Plotting of data
- viewing raw data
- Collects Cross over data

In [4]:
import plotly.graph_objects as go

# Calculate a {days_average}-day moving average for illustration
days_average = 20
df["Moving Average"] = df["close"].rolling(window=days_average).mean()
df["Moving Average 50"] = df["close"].rolling(window=50).mean()
df["Moving Average 200"] = df["close"].rolling(window=200).mean()


# Create candlestick chart
fig = go.Figure(data=[
    go.Candlestick(x=df.index, 
                   open=df["open"], high=df["high"], low=df["low"], close=df["close"],
                   name="BTC Price"),
    # Add moving average line
    go.Scatter(x=df.index, y=df["Moving Average"], mode="lines", line=dict(color="#2111cc", width=1),
               name=f"{days_average}-day MA"),
    go.Scatter(x=df.index, y=df["Moving Average 50"], mode="lines", line=dict(color="#ffbc2c", width=1),
               name=f"{50}-day MA"),
    go.Scatter(x=df.index, y=df["Moving Average 200"], mode="lines", line=dict(color="#1bc7bb", width=1),
               name=f"{200}-day MA")
])
fig.update_layout(title=f"BTC/USDT Daily Price with {days_average}-day Moving Average",
                  yaxis_title="Price (USD)", xaxis_title="Date")
fig.update_xaxes(rangeslider_visible=False)  # disable range slider for clarity
fig.show()


Data Collection: Stores the crossing positions of 3 moving averages

In [5]:
## Collecting All Cross Data at different Time Periods
pairs = [("Moving Average", "Moving Average 50"), ("Moving Average", "Moving Average 200"),
         ("Moving Average 50", "Moving Average 200")]
intersections = []

for a, b in pairs:
    cond = df[a] > df[b]
    cross = cond != cond.shift(1)
    pts = df.loc[cross & cross.notna(), [a, b]].copy()
    pts["pair"] = f"{a} vs {b}"
    pts["time"] = pts.index
    intersections.append(pts)

df_cross = pd.concat(intersections).sort_values("time").reset_index(drop=True)
df_cross.head(5)


# Filtering Data
df_cross1 = df_cross[df_cross['pair']=='Moving Average vs Moving Average 50'].reset_index(drop=True)
df_cross2 = df_cross[df_cross['pair']=='Moving Average vs Moving Average 200'].reset_index(drop=True)
df_cross3 = df_cross[df_cross['pair']=='Moving Average 50 vs Moving Average 200'].reset_index(drop=True)
df_cross1.head(5)
# df_cross2.head(5)
# df_cross3.head(5)

,Moving Average,Moving Average 50,pair,time,Moving Average 200
0,NaN,NaN,Moving Average vs Moving Average 50,2017-08-17,NaN
1,4257.6315,4222.1386,Moving Average vs Moving Average 50,2017-10-11,NaN
2,14312.2545,14425.7598,Moving Average vs Moving Average 50,2018-01-16,NaN
3,10387.4700,10267.6060,Moving Average vs Moving Average 50,2018-03-04,NaN
4,9413.1740,9438.4350,Moving Average vs Moving Average 50,2018-03-21,NaN


### 3. Define Trading Strategies: Martingale vs. "Fish Tail" (Anti-Martingale)

In [6]:
## INPUT Set Range of Data Here =====================================
deposit_date  = "2021-01-01"
horizon_end   = "2025-08-10"
initial_deposit = 1000

In [7]:
# Martingale strategy simulation
def simulate_martingale(data, base_position=1, max_adds=5):
    """
    Simulate a Martingale strategy on the given price data.
    - base_position: initial trade size (in units)
    - max_adds: max number of times to double-down (to limit risk)
    Returns: lists of buy events, sell events, and total profit.
    """
    balance = 0.0       # cumulative profit/loss
    positions = 0.0      # current open position (in units of asset)
    avg_cost = 0.0       # average cost of current position
    buys = []            # list of (time, price, size) for buy actions
    sells = []           # list of (time, price, size) for sell actions
    add_count = 0        # how many times we've added (doubled down) in current cycle
    
    for time, price in data["close"].items():  # iterate over each day (timestamp and closing price)
        if positions == 0:
            # No open position -> start a new trade
            positions = base_position
            avg_cost = price
            balance -= price * base_position  # spending money to buy
            buys.append((time, price, base_position))
            add_count = 0
            # (We open a position and wait at least until next iteration to decide further actions)
            continue
        
        # If we have an open position, check price movement relative to avg_cost
        if price > avg_cost:
            # Price went up above average cost -> take profit (sell all)
            balance += price * positions  # money received from selling
            sells.append((time, price, positions))
            positions = 0.0
            avg_cost = 0.0
            add_count = 0
            # Position closed; we will open a new one on the next loop iteration
        elif price < avg_cost:
            # Price went down -> it's a losing position now
            if add_count < max_adds:
                # Double down (buy more to lower average cost)
                additional = positions  # buy amount equal to current total position
                positions += additional
                # Update average cost of the position after buying at this price
                avg_cost = (avg_cost * (positions - additional) + price * additional) / positions
                balance -= price * additional  # spend more to buy
                buys.append((time, price, additional))
                add_count += 1
                # (We continue holding; do not close position yet)
            # If max_adds reached, we do nothing further (hold and hope for recovery)
    # If loop ends while still holding a position, close it at the last price
    if positions > 0:
        balance += price * positions
        sells.append((time, price, positions))
        positions = 0.0
    return buys, sells, balance



# Fish Tail (Anti-Martingale) strategy simulation
def simulate_antimartingale(data, base_position=1, max_adds=5, target_wins=3):
    """
    Simulate an Anti-Martingale (Fish Tail) strategy on the given data.
    - base_position: initial trade size
    - max_adds: max number of times to double on wins
    - target_wins: take profit after this many consecutive wins (up days)
    Returns: lists of buy events, sell events, and total profit.
    """
    balance = 0.0
    positions = 0.0
    avg_cost = 0.0
    buys = []
    sells = []
    win_streak = 0    # count of consecutive winning days in current run
    add_count = 0
    
    prev_price = None  # to track previous day's price for win/loss determination
    for time, price in data["close"].items():
        if positions == 0:
            # No position -> open a new base position
            positions = base_position
            avg_cost = price
            balance -= price * base_position
            buys.append((time, price, base_position))
            win_streak = 0
            add_count = 0
            prev_price = price
            continue
        
        # Determine if today is a "win" (price went up) or "loss" (price went down)
        if price > prev_price:
            # Price went up (favorable move)
            win_streak += 1
            if win_streak >= target_wins:
                # Achieved target consecutive wins -> take profit
                balance += price * positions
                sells.append((time, price, positions))
                positions = 0.0
                avg_cost = 0.0
                win_streak = 0
                add_count = 0
                # Position closed; will open new on next iteration
            else:
                # Not yet at target, so increase position (pyramid in)
                if add_count < max_adds:
                    additional = positions  # double current position
                    positions += additional
                    avg_cost = (avg_cost * (positions - additional) + price * additional) / positions
                    balance -= price * additional
                    buys.append((time, price, additional))
                    add_count += 1
                # If max_adds reached, just hold and continue the streak
        else:
            # Price did not go up (flat or down day -> treat as end of winning streak)
            balance += price * positions  # sell all at current price (could be a loss or smaller profit)
            sells.append((time, price, positions))
            positions = 0.0
            avg_cost = 0.0
            win_streak = 0
            add_count = 0
            # (Will open a new position next iteration)
        prev_price = price  # update previous day's price
    # If still holding at end, close position at last price
    if positions > 0:
        balance += price * positions
        sells.append((time, price, positions))
    return buys, sells, balance

# Run simulations on our historical DataFrame
martingale_buys, martingale_sells, martingale_profit = simulate_martingale(df)
fish_buys, fish_sells, fish_profit = simulate_antimartingale(df)
print(f"Martingale Strategy - Final P&L: {martingale_profit:.2f} USD")
print(f"Fish Tail Strategy - Final P&L: {fish_profit:.2f} USD")


Martingale Strategy - Final P&L: 1951676.55 USD
Fish Tail Strategy - Final P&L: 17449.07 USD


Martingale System (Modified with Initial Deposit)
- added initial deposit feature
- added specify timeframe option
- added profit and loss estimate within timeframe
- uses closed price values as the avg cost

Output Summary
- Balance goes way too low. Balance goes negative.

In [8]:
import pandas as pd

def simulate_martingale(
    data: pd.DataFrame,
    base_position=1.0,
    max_adds=5,
    initial_deposit=2000.0,
):
    """
    Simulate a Martingale strategy on a Series/DataFrame with a 'close' column.
    Returns:
        buys:  [(time, price, size), ...]
        sells: [(time, price, size), ...]
        net_profit: float
        final_balance: float
        initial_deposit: float
        equity_df: pd.DataFrame indexed by time with column 'equity' (cash + mark-to-market)
    Notes:
        - This version allows the balance to go negative (no margin checks).
        - 'data' should be sliced to your desired test window before passing in.
    """
    close = data["close"] # setting price data as the close prices from extracted data
    # close = data["Moving Average"]
    balance = float(initial_deposit)
    positions = 0.0
    avg_cost = 0.0
    buys, sells = [], []
    add_count = 0
    equity_curve = []

    prev_price = None
    for time, price in close.items():
        # Record equity before today’s action (mark-to-market) for smoother curve
        mtm = balance + positions * price
        equity_curve.append((time, mtm, balance))

        if positions == 0.0:
            # Open new base position
            positions = float(base_position)
            avg_cost = price
            balance -= price * positions
            buys.append((time, price, positions))
            add_count = 0
        else:
            if price > avg_cost:
                # Take profit: close all
                balance += price * positions
                sells.append((time, price, positions))
                positions = 0.0
                avg_cost = 0.0
                add_count = 0
            elif price < avg_cost and add_count < max_adds: #
                # Double down
                additional = positions
                positions += additional
                avg_cost = (avg_cost * (positions - additional) + price * additional) / positions
                balance -= price * additional
                buys.append((time, price, additional))
                add_count += 1

        prev_price = price

    # Close any remaining position at last price
    if positions > 0:
        balance += price * positions
        sells.append((time, price, positions))

    net_profit = balance - initial_deposit
    equity_df = pd.DataFrame(equity_curve, columns=["time", "equity","Balance"]).set_index("time")
    return buys, sells, net_profit, balance, initial_deposit, equity_df


from datetime import timedelta

# def forward_test_actual(df, start_date, horizon_days=180, **martingale_kwargs):
def forward_test_actual(df, start_date, horizon_days, **martingale_kwargs):
    """
    Slice actual data from start_date to start_date + horizon_days,
    run Martingale, and return the results + actual equity curve.
    """
    start_date = pd.to_datetime(start_date)
    # end_date = start_date + pd.Timedelta(days=horizon_days)
    end_date = horizon_days

    df_slice = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    if df_slice.empty or len(df_slice) < 5:
        raise ValueError("Not enough data in the requested forward window.")
    return simulate_martingale(df_slice, **martingale_kwargs)

# RUNNING CODE INPUTS
# deposit_date = "2025-01-1"   # <-- choose when you 'start' with your initial deposit
# horizon_days = "2025-08-10"
# deposit_date  = "2024-01-01"
# horizon_end   = "2025-08-10"
# horizon_days = 500
# initial_deposit = 500.0 # Amount to deposit

(buys_act, sells_act, pnl_act, final_bal_act, init_dep_act, equity_act
) = forward_test_actual(
    df, start_date=deposit_date, horizon_days=horizon_end,
    base_position=0.01,   # size in BTC (example) of your very first Buy when you open the position. % of the actual Bitcoin i will actually buy
    max_adds=5, # How many times to allow doubling down
    initial_deposit=initial_deposit
)

print(f"[ACTUAL] From {deposit_date} to {horizon_end}")
print(f"Initial Deposit: ${init_dep_act:,.2f}")
print(f"Final Balance : ${final_bal_act:,.2f}")
print(f"Net Profit    : ${pnl_act:,.2f}")
print(f"Min Equity    : ${equity_act['equity'].min():,.2f}") # The lowest value your account equity reached at any point during the simulation period.
print(f"Profit Increase  : {final_bal_act/init_dep_act:,.2f} times increase")


## add drawdown. What are the positions where your trade performns loss

[ACTUAL] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance : $20,338.42
Net Profit    : $19,338.42
Min Equity    : $-2,108.65
Profit Increase  : 20.34 times increase


Martingale System (Spot Trading)
- Balance not allowed to go below 0

In [9]:
import pandas as pd

def simulate_martingale_spot(
    data: pd.DataFrame,
    base_position=1.0,        # in BTC
    max_adds=5,
    initial_deposit=2000.0,
    fee_rate=0.0,             # e.g., 0.001 = 0.1% per trade
    allow_partial_buy=True,   # if False, skip buys you can't fully afford
    min_notional_usd=5.0      # skip/clip tiny buys under this notional
):
    """
    Martingale with spot-style cash constraints:
    - Balance (cash) never goes negative.
    - If you can't afford the next buy, either skip or buy a smaller amount (configurable).
    - Equity still = balance + positions * price (mark-to-market).
    """
    close = data["close"]
    # close = data["Moving Average"]
    # close = data["Moving Average 200"]
    # close = data["Moving Average"]

    balance = float(initial_deposit)  # CASH ONLY; never goes < 0
    positions = 0.0                   # BTC held
    avg_cost = 0.0
    buys, sells = [], []
    add_count = 0
    equity_curve = []

    def affordable_btc_to_buy(price, desired_btc):
        """Return BTC size you can afford (after fees) without making balance negative."""
        if desired_btc <= 0:
            return 0.0
        # cash needed including fee
        desired_cost = price * desired_btc
        desired_fee  = desired_cost * fee_rate
        total_needed = desired_cost + desired_fee
        if total_needed <= balance:
            return desired_btc  # can fully afford
        if not allow_partial_buy:
            return 0.0
        # Clip down to affordable amount
        # Solve for x: balance >= x*price + fee_rate*(x*price) = x*price*(1+fee_rate)
        x = balance / (price * (1.0 + fee_rate))
        # avoid dust/notional too small
        if (x * price) < min_notional_usd:
            return 0.0
        return x

    for time, price in close.items():
        # Record equity BEFORE action (mark-to-market). You can move this below actions if preferred.
        mtm = balance + positions * price
        equity_curve.append((time, mtm, balance))

        if positions == 0.0:
            # Try to open initial position within cash constraints
            buy_btc = affordable_btc_to_buy(price, float(base_position))
            if buy_btc > 0.0:
                cost = buy_btc * price
                fee  = cost * fee_rate
                balance -= (cost + fee)              # cash out
                positions += buy_btc
                avg_cost = (avg_cost * 0 + price * buy_btc) / (buy_btc if buy_btc else 1)
                buys.append((time, price, buy_btc))
                add_count = 0
            # else: can't afford base buy → just wait for next bar
        else:
            if price > avg_cost:
                # Take profit: close all (pay fee on proceeds)
                proceeds = positions * price
                fee = proceeds * fee_rate
                balance += (proceeds - fee)
                sells.append((time, price, positions))
                positions = 0.0
                avg_cost = 0.0
                add_count = 0
            elif price < avg_cost and add_count < max_adds:
                # Double-down DESIRED size = current position
                desired = positions
                buy_btc = affordable_btc_to_buy(price, desired)
                if buy_btc > 0.0:
                    cost = buy_btc * price
                    fee  = cost * fee_rate
                    balance -= (cost + fee)
                    # update avg cost with weighted average
                    new_pos = positions + buy_btc
                    avg_cost = (avg_cost * positions + price * buy_btc) / new_pos
                    positions = new_pos
                    buys.append((time, price, buy_btc))
                    # Only count as an "add" if we actually increased position meaningfully
                    if buy_btc >= 0.999999 * desired:
                        add_count += 1
                # else: cannot afford (or too small) → skip this add and keep holding

    # Close any remaining position at last price (optional; many sims do this)
    if positions > 0:
        proceeds = positions * price
        fee = proceeds * fee_rate
        balance += (proceeds - fee)
        sells.append((time, price, positions))
        positions = 0.0

    net_profit = balance - initial_deposit
    equity_df = pd.DataFrame(equity_curve, columns=["time", "equity","Balance"]).set_index("time")
    return buys, sells, net_profit, balance, initial_deposit, equity_df

def forward_test_actual(df, start_date, horizon_days, **martingale_kwargs):
    """
    Slice actual data from start_date to start_date + horizon_days,
    run Martingale, and return the results + actual equity curve.
    """
    start_date = pd.to_datetime(start_date)
    # end_date = start_date + pd.Timedelta(days=horizon_days)
    end_date = horizon_days

    df_slice = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    if df_slice.empty or len(df_slice) < 5:
        raise ValueError("Not enough data in the requested forward window.")
    return df_slice

# RUNNING CODE INPUTS =========================================================
# deposit_date = "2021-01-1"   # <-- choose when you 'start' with your initial deposit
# horizon_end = "2025-08-10"
# horizon_end = 500
# initial_deposit = 500.0 # Amount to deposit
base_position = 0.00060  # size in BTC (example) of your very first Buy when you open the position. % of the actual Bitcoin i will actually buy

df_slice = forward_test_actual(
    df, start_date=deposit_date, horizon_days=horizon_end,
    base_position=base_position,  
    max_adds=5, # How many times to allow doubling down
    initial_deposit=initial_deposit
)

(buys_act, sells_act, pnl_act, final_bal_act, init_dep_act, equity_act
) = simulate_martingale_spot(
    df_slice,
    base_position=base_position,
    max_adds=5,
    initial_deposit=initial_deposit,
    fee_rate=0.00,            # 0.1% fee example
    allow_partial_buy=True,
    min_notional_usd=10.0
)

print(f"[ACTUAL] From {deposit_date} to {horizon_end}")
print(f"Initial Deposit: ${init_dep_act:,.2f}")
print(f"Final Balance : ${final_bal_act:,.2f}")
print(f"Net Profit    : ${pnl_act:,.2f}")
print(f"Min Equity    : ${equity_act['equity'].min():,.2f}") # The lowest value your account equity reached at any point during the simulation period.
print(f"Profit Increase  : {final_bal_act/init_dep_act:,.2f} times increase")




[ACTUAL] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance : $2,154.67
Net Profit    : $1,154.67
Min Equity    : $813.48
Profit Increase  : 2.15 times increase


Buy All in and Hold
- Assuming common way to buy stocks without going through any complicated trading algorithm

In [10]:
# import pandas as pd
from datetime import datetime, timedelta

# === SIMPLE BUY & HOLD =======================================================
def simulate_all_in_hold(
    data: pd.DataFrame,
    initial_deposit=2000.0,
    fee_rate=0.0,          # e.g., 0.001 = 0.1% per trade
    sell_at_end=True       # if True, convert BTC back to USD at final close
):
    """
    All-in & Hold:
      - Buy BTC with (almost) all cash at the first bar (pay fee).
      - Hold BTC to the end.
      - Equity = cash + position * price (mark-to-market each bar).
      - Optionally sell at the end to realize USD.
    Expects data with a 'close' column and a DatetimeIndex.
    """
    close = data["close"].astype(float)

    # State
    cash = float(initial_deposit)
    btc  = 0.0
    equity_curve = []

    # Buy at the first bar
    first_price = close.iloc[0]
    # amount of BTC we can buy considering fee on notional
    # cash >= qty*price*(1+fee)  -> qty = cash / (price*(1+fee))
    qty = cash / (first_price * (1.0 + fee_rate))
    cost = qty * first_price
    fee  = cost * fee_rate
    cash -= (cost + fee)
    btc   = qty

    # Track equity over time
    for t, px in close.items():
        equity = cash + btc * px
        equity_curve.append((t, equity, cash, btc, px))

    # Optionally sell at the end
    final_price = close.iloc[-1]
    if sell_at_end and btc > 0:
        proceeds = btc * final_price
        fee = proceeds * fee_rate
        cash += (proceeds - fee)
        btc = 0.0

    net_profit = cash - initial_deposit
    equity_df = pd.DataFrame(equity_curve, columns=["time","equity","cash","btc","price"]).set_index("time")
    return {
        "net_profit": net_profit,
        "final_balance": cash,
        "initial_deposit": initial_deposit,
        "btc_units_held": btc,            # 0 if sold at end
        "buy_price": first_price,
        "sell_price": final_price if sell_at_end else None,
        "equity_curve": equity_df
    }

# === WINDOW SLICE (cleaned) ==================================================
def forward_test_actual(df, start_date, end_date):
    """
    Slice data between start_date and end_date (inclusive).
    - start_date / end_date can be 'YYYY-MM-DD' strings or pandas Timestamps.
    """
    start_date = pd.to_datetime(start_date)
    end_date   = pd.to_datetime(end_date)
    df_BuynHold = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    if df_BuynHold.empty or len(df_BuynHold) < 2:
        raise ValueError("Not enough data in the requested window.")
    return df_BuynHold

# === RUNNING CODE INPUTS =====================================================
# Assumes you already have `df` with a DatetimeIndex and columns including 'close'
# Example:
# df.index = pd.to_datetime(df.index, utc=True)  # ensure datetime index
# df['close'] = df['close'].astype(float)

deposit_date_BuynHold  = "2024-01-01"
horizon_end_BuynHold   = "2025-08-10"       # end date instead of "horizon_days"
# initial_deposit = 1000.0
fee_rate = 0.000  # 0.1% example

# Slice the backtest window
df_BuynHold = forward_test_actual(df, start_date=deposit_date, end_date=horizon_end)

# Run all-in & hold
res = simulate_all_in_hold(
    df_BuynHold,
    initial_deposit=initial_deposit,
    fee_rate=fee_rate,
    sell_at_end=True
)

eq = res["equity_curve"]

print(f"[BUY & HOLD] From {deposit_date} to {horizon_end}")
print(f"Initial Deposit: ${res['initial_deposit']:,.2f}")
print(f"Final Balance  : ${res['final_balance']:,.2f}")
print(f"Net Profit     : ${res['net_profit']:,.2f}")
print(f"Min Equity     : ${eq['equity'].min():,.2f}")
print(f"Buy @ {res['buy_price']:,.2f} | Sell @ {res['sell_price']:,.2f}")
print(f"Profit Increase  : {res['final_balance']/res['initial_deposit']:,.2f} times increase")


[BUY & HOLD] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance  : $4,067.07
Net Profit     : $3,067.07
Min Equity     : $538.03
Buy @ 29,331.69 | Sell @ 119,294.01
Profit Increase  : 4.07 times increase


MA Crossover Strategy
- Start Trade. Buy Signals. MA50 crosses above MA200 (initial/long buy signal). MA20 crossing above MA50 (subsequent buy signals)
- Close Position Signal. MA20 crosses below MA200 (close all).

In [11]:
import pandas as pd

def simulate_all_in_crossover(
    data: pd.DataFrame,
    price_col="close",
    fast_col="Moving Average",         # e.g., MA20
    slow_col="Moving Average 50",      # e.g., MA50
    initial_deposit=1000.0,
    fee_rate=0.000,                    # 0.1% per trade
    sell_at_end=True,                  # exit any open position at the final bar
    execute_on="close"                 # "close" (immediate on the cross bar)
):
    """
    Immediate-entry crossover (long-only, all-in):
      - BUY 100% when fast MA crosses ABOVE slow MA (bullish cross) on that bar's close.
      - SELL 100% when fast MA crosses BELOW slow MA (bearish cross) on that bar's close.
      - Equity marked-to-market each bar.
    Expects 'data' with a DateTimeIndex and columns: price_col, fast_col, slow_col.
    """

    df = data.copy().sort_index()
    for c in [price_col, fast_col, slow_col]:
        if c not in df.columns:
            raise ValueError(f"Column '{c}' not found in data.")
    df = df[[price_col, fast_col, slow_col]].dropna()

    px   = df[price_col].astype(float)
    fast = df[fast_col].astype(float)
    slow = df[slow_col].astype(float)

    # Signal state: 1 when fast > slow else 0
    cond = (fast > slow).astype(int)

    # Immediate-entry crosses on the SAME bar:
    cross_up   = (cond.diff() ==  1)   # bullish cross this bar
    cross_down = (cond.diff() == -1)   # bearish cross this bar

    cash = float(initial_deposit)
    qty  = 0.0
    equity_curve = []
    trades = []

    for t in df.index:
        price_here = px.loc[t]

        # --- Trade first (immediate execution on this bar's close) ---
        if cross_up.loc[t] and qty == 0.0:
            # All-in buy at current close (account for fee on notional)
            # cash >= qty * price * (1+fee) -> qty = cash / (price*(1+fee))
            buy_qty = cash / (price_here * (1.0 + fee_rate))
            notional = buy_qty * price_here
            fee = notional * fee_rate
            cash -= (notional + fee)
            qty  += buy_qty
            trades.append({"time": t, "side": "BUY", "price": price_here, "qty": buy_qty, "fee": fee})

        elif cross_down.loc[t] and qty > 0.0:
            # Full exit at current close (fee on proceeds)
            proceeds = qty * price_here
            fee = proceeds * fee_rate
            cash += (proceeds - fee)
            trades.append({"time": t, "side": "SELL", "price": price_here, "qty": qty, "fee": fee})
            qty = 0.0

        # --- Mark-to-market after any trade on this bar ---
        equity = cash + qty * price_here
        equity_curve.append((t, equity, cash, qty, price_here))

    # Optional: close at the very end
    if sell_at_end and qty > 0.0:
        t = df.index[-1]
        price_here = px.loc[t]
        proceeds = qty * price_here
        fee = proceeds * fee_rate
        cash += (proceeds - fee)
        trades.append({"time": t, "side": "SELL", "price": price_here, "qty": qty, "fee": fee})
        qty = 0.0
        # update last equity snapshot
        equity_curve[-1] = (t, cash, cash, qty, price_here)

    equity_df = pd.DataFrame(equity_curve, columns=["time","equity","cash","qty","price"]).set_index("time")
    trades_df = pd.DataFrame(trades)

    result = {
        "equity_curve":   equity_df,
        "trades":         trades_df,
        "final_balance":  cash,
        "initial_deposit": initial_deposit,
        "net_profit":     cash - initial_deposit,
        "fast_col":       fast_col,
        "slow_col":       slow_col
    }
    return result


In [12]:
# Use your existing df with DateTimeIndex and columns: 'close', 'Moving Average', 'Moving Average 50', 'Moving Average 200'

# deposit_date  = "2024-01-01"
# horizon_end   = "2025-08-10"
fee_rate      = 0.000
initial_dep   = initial_deposit

df_cross_Allin = forward_test_actual(df, start_date=deposit_date, end_date=horizon_end)

# Example 1: trade MA (20) vs MA50
res_20_50 = simulate_all_in_crossover(
    df_cross_Allin,
    price_col="close",
    fast_col="Moving Average",
    slow_col="Moving Average 50",
    initial_deposit=initial_dep,
    fee_rate=fee_rate,
    sell_at_end=True
)

# Example 2: trade MA50 vs MA200 (golden/death cross)
res_50_200 = simulate_all_in_crossover(
    df_cross_Allin,
    price_col="close",
    fast_col="Moving Average 50",
    slow_col="Moving Average 200",
    initial_deposit=initial_dep,
    fee_rate=fee_rate,
    sell_at_end=True
)

# Example 2: trade MA50 vs MA200 (golden/death cross)
res_20_200 = simulate_all_in_crossover(
    df_cross_Allin,
    price_col="close",
    fast_col="Moving Average",
    slow_col="Moving Average 200",
    initial_deposit=initial_dep,
    fee_rate=fee_rate,
    sell_at_end=True
)

for name, res in [("MA20/50", res_20_50), ("MA50/200", res_50_200),("MA20/200", res_20_200)]:
    eq = res["equity_curve"]
    print(f"\n[{name}] From {deposit_date} to {horizon_end}")
    print(f"Initial Deposit: ${res['initial_deposit']:,.2f}")
    print(f"Final Balance: ${res['final_balance']:,.2f}")
    print(f"Net PnL: ${res['net_profit']:,.2f}")
    print(f"Min Eq : ${eq['equity'].min():,.2f}")
    print(f"Trades : {len(res['trades'])}")
    print(f"Profit Increase  : {res['final_balance']/res['initial_deposit']:,.2f} times increase")
    # if not res["trades"].empty:
    #     print(res["trades"].head())



[MA20/50] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance: $1,508.72
Net PnL: $508.72
Min Eq : $557.02
Trades : 38
Profit Increase  : 1.51 times increase

[MA50/200] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance: $2,124.76
Net PnL: $1,124.76
Min Eq : $775.85
Trades : 10
Profit Increase  : 2.12 times increase

[MA20/200] From 2021-01-01 to 2025-08-10
Initial Deposit: $1,000.00
Final Balance: $2,655.45
Net PnL: $1,655.45
Min Eq : $741.20
Trades : 12
Profit Increase  : 2.66 times increase


Martingale x MA Cross Hybrid Strategy
- Start Trade. Buy Signals. MA50 crosses above MA200 (initial/long buy signal). MA20 crossing above MA50 (subsequent buy signals)
- Close Position Signal. MA20 crosses below MA200 (close all).

### 4. Plot Trading Signals on the Price Chart

Plots The Timeframe and Personal Equity Growing overtime as the trade goes on

In [13]:
df.head()

,open,high,low,close,volume,Moving Average,Moving Average 50,Moving Average 200
open_time,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,NaN,NaN
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,NaN,NaN
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,NaN,NaN


In [16]:
# ---- PLOT: Candlesticks + Buy/Sell markers + Equity curve (secondary y-axis) ----
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# 1) Build the same price window used for the forward test
window = df.loc[pd.to_datetime(deposit_date):pd.to_datetime(horizon_end)].copy()

# 2) Extract Buy/Sell points from your simulation outputs
buy_x = [t for (t, p, s) in buys_act]
buy_y = [p for (t, p, s) in buys_act]
sell_x = [t for (t, p, s) in sells_act]
sell_y = [p for (t, p, s) in sells_act]

# 3) Make a dual-axis figure: price on left, equity on right
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Candlesticks (primary y)
fig.add_trace(
    go.Candlestick(
        x=window.index,
        open=window["open"], high=window["high"], low=window["low"], close=window["close"],
        name=f"{symbol}"
    ),
    secondary_y=False
)

# 4) Buy/Sell markers on price
if buy_x:
    fig.add_trace(
        go.Scatter(
            x=buy_x, y=buy_y,
            mode="markers",
            marker_symbol="triangle-up",
            marker_size=10,
            name="Buy",
            hovertemplate="Buy<br>%{x|%Y-%m-%d}<br>Price: %{y:.2f}<extra></extra>"
        ),
        secondary_y=False
    )

if sell_x:
    fig.add_trace(
        go.Scatter(
            x=sell_x, y=sell_y,
            mode="markers",
            marker_symbol="triangle-down",
            marker_size=10,
            name="Sell",
            hovertemplate="Sell<br>%{x|%Y-%m-%d}<br>Price: %{y:.2f}<extra></extra>"
        ),
        secondary_y=False
    )

# 5) Moving Average (secondary y)
fig.add_trace(
    go.Scatter(
        x=window.index,
        y=window["Moving Average 200"],
        mode="lines",
        name="Moving Average",
        hovertemplate="Moving Average<br>%{x|%Y-%m-%d}<br>$%{y:,.2f}<extra></extra>",
        line = dict(width=1, color = "#A679E0"),

    ),
    secondary_y=False
)

# 5) Moving Average (secondary y)
fig.add_trace(
    go.Scatter(
        x=window.index,
        y=window["Moving Average 50"],
        mode="lines",
        name="Moving Average 50",
        hovertemplate="Moving Average 50<br>%{x|%Y-%m-%d}<br>$%{y:,.2f}<extra></extra>",
        line = dict(width=1, color = "#C461CF"),

    ),
    secondary_y=False
)

# 5) Moving Average (secondary y)
fig.add_trace(
    go.Scatter(
        x=window.index,
        y=window["Moving Average"],
        mode="lines",
        name="Moving Average 200",
        hovertemplate="Moving Average 200<br>%{x|%Y-%m-%d}<br>$%{y:,.2f}<extra></extra>",
        line = dict(width=1, color = "#75387D"),

    ),
    secondary_y=False
)

# 5) Equity curve (secondary y)
fig.add_trace(
    go.Scatter(
        x=equity_act.index,
        y=equity_act["equity"],
        mode="lines",
        name="Equity",
        hovertemplate="Equity<br>%{x|%Y-%m-%d}<br>$%{y:,.2f}<extra></extra>",
        line = dict(width=1)
    ),
    secondary_y=True
)

# 5) Balance curve (secondary y)
fig.add_trace(
    go.Scatter(
        x=equity_act.index,
        y=equity_act["Balance"],
        mode="lines",
        name="Balance",
        hovertemplate="Balance<br>%{x|%Y-%m-%d}<br>$%{y:,.2f}<extra></extra>",
        line = dict(width=0.5, color = "#EDB437")

    ),
    secondary_y=True
)

# 6) Cross Mark =======================================================
fig.add_trace(
        go.Scatter(
            # x=df_cross1['time'], y=df_cross1['Moving Average'], # INPUT. Plot MA20 Crossing MA50
            # x=df_cross2['time'], y=df_cross2['Moving Average'], # INPUT. Plot MA20 Crossing MA200
            x=df_cross3['time'], y=df_cross3['Moving Average 50'],# INPUT. Plot MA50 Crossing MA200
            mode="markers",
            marker_symbol="x",
            marker_size=10,
            name="MA Cross",
            hovertemplate="MA Cross<br>%{x|%Y-%m-%d}<br>Price: %{y:.2f}<extra></extra>",
            marker = dict(color = "#3EA0EE")
        ),
        secondary_y=False
    )

# 6) Cosmetics: titles, axes, helpful guides
fig.update_layout(
    # title=f"Martingale • Candlesticks + Trades + Equity  |  {deposit_date} → {horizon_days}",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    hovermode="x unified"
)
fig.update_xaxes(title_text="Date", rangeslider_visible=False)

fig.update_yaxes(title_text=f"{symbol} Price", secondary_y=False)
fig.update_yaxes(title_text="Equity and Balance (USD)", secondary_y=True)

# Add a vertical line at the deposit/start date
fig.add_vline(x=pd.to_datetime(deposit_date), line_dash="dot", opacity=0.5)

# Optional: zero-equity reference line on the right axis
fig.add_hline(y=0, line_dash="dash", opacity=0.3, secondary_y=True)

fig.show()


In [15]:
equity_act

,equity,Balance
time,,
2021-01-01,1000.000000,1000.000000
2021-01-02,1001.707984,982.400986
2021-01-03,1001.707984,1001.707984
2021-01-04,1001.101180,981.907954
2021-01-05,1003.454164,962.714728
...,...,...
2025-08-06,2148.605377,2010.614653
2025-08-07,2148.605377,2148.605377
2025-08-08,2148.127015,2078.122171


### Summary of Strategies